In [4]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1)

# DuckDuckGoSearchAPIWrapper 인스턴스를 한 번만 생성
ddg = DuckDuckGoSearchAPIWrapper()

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for. Example query: Stock Market Symbol for Apple Company"
    )

class StockMarketSymbolSearchTool(BaseTool):
    name: str = "StockMarketSymbolSearchTool"  # 타입 명시
    description: str = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query: str) -> str:
        return ddg.run(query)  # DuckDuckGo API를 재사용

# 에이전트 초기화
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[StockMarketSymbolSearchTool()],
)

prompt = "Give me information on 삼성전자's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

response = agent.invoke(prompt)
print(response)




> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for 삼성전자'}`


See the latest Samsung Electronics Co Ltd stock price (005930:XKRX), related news, valuation, dividends and more to help you make your investing decisions. Real-time stock price information. Investor Relations ; Stock Information; Stock Chart; open close About Us. Leadership & Mission - Our mission & Values - Business Principles - Our CEO - History; IR General Information; open close Financial Information & Disclosures. Financial Highlights; Get the latest Samsung Electronics Co., Ltd. (KRX:005930) stock price quote with financials, statistics, dividends, charts and more. Get undefined (005930.KS) real-time stock quotes, news, price and financial information from Reuters to inform your trading and investments SAMSUNG ELECTRONICS CO LTD (ATT) SMSN Company page - Search stock, chart, recent trades, company information, trading information, company news, fund